In [1]:
#data:kaggle competitions download -c google-quest-challenge
%ls

������ C �еľ��� Local Disk
 �������к��� 025D-76D5

 c:\Users\rukam\source\repos\FLIP01\Code ��Ŀ¼

2020/01/14  23:02    <DIR>          .
2020/01/14  23:02    <DIR>          ..
2020/01/06  20:27                 0 project.ipynb
2020/01/07  15:38         4,154,855 start-from-here-quest-complete-eda-fe.ipynb
2020/01/14  23:17             4,921 ti.ipynb
               3 ���ļ�      4,159,776 �ֽ�
               2 ��Ŀ¼ 31,974,793,216 �����ֽ�


In [19]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
train_set = pd.read_csv('Data/train.csv')
test_set =  pd.read_csv('Data/test.csv')
submmistion=pd.read_csv('Data/sample_submission.csv')

FileNotFoundError: [Errno 2] File b'Data/train.csv' does not exist: b'Data/train.csv'

In [17]:
def get_entropy(df, round_val=3):
    result = []
    for col in df.columns:
        data = df[col].value_counts(normalize=True)
        result.append(round(stats.entropy(data, base=2), round_val))
    return result
def get_summary(df, cols=None):
    summary = pd.DataFrame()
    vcs = pd.DataFrame()
    if cols == None:
        cols = df.columns.values
    summary['Name'] = cols
    summary['dtype'] = df[cols].dtypes.values
    summary['Unique'] = df[cols].nunique().values
    summary['Missing'] = df[cols].isnull().sum().values
    summary['Mode'] = df[cols].mode().iloc[0].values
    summary['Mode Freq'] = df[cols].apply(lambda ser:pd.value_counts(ser, normalize=True).values[0]).values
    summary['Entropy'] = get_entropy(df[cols])
    return summary

In [5]:
train_set = pd.read_csv('../Data/train.csv')
test_set =  pd.read_csv('../Data/test.csv')
submmistion=pd.read_csv('../Data/sample_submission.csv')

In [13]:
data_cols = train_set.columns[:11]
target_cols = train_set.columns[11:]

In [14]:
data_cols

Index(['qa_id', 'question_title', 'question_body', 'question_user_name',
       'question_user_page', 'answer', 'answer_user_name', 'answer_user_page',
       'url', 'category', 'host'],
      dtype='object')

In [20]:
summary_data = get_summary(train_set[data_cols])
summary_data

,Name,dtype,Unique,Missing,Mode,Mode Freq,Entropy
0,qa_id,int64,6079,0,0,0.000165,12.570
1,question_title,object,3583,0,What is the best introductory Bayesian statist...,0.001974,11.641
2,question_body,object,3583,0,Which is the best introductory textbook for Ba...,0.001974,11.641
3,question_user_name,object,3215,0,Mike,0.002468,11.420
4,question_user_page,object,3422,0,https://stats.stackexchange.com/users/5,0.001974,11.557
5,answer,object,6079,0,\n\nPrint it and give it to your teacher. Or s...,0.000165,12.570
6,answer_user_name,object,4114,0,Scott,0.003290,11.678
7,answer_user_page,object,4430,0,https://tex.stackexchange.com/users/4427,0.002797,11.846
8,url,object,3583,0,http://stats.stackexchange.com/questions/125/w...,0.001974,11.641
9,category,object,5,0,TECHNOLOGY,0.401546,2.144


In [21]:
summary_target = get_summary(train_set[target_cols])
summary_target

,Name,dtype,Unique,Missing,Mode,Mode Freq,Entropy
0,question_asker_intent_understanding,float64,9,0,1.000000,0.466031,2.020
1,question_body_critical,float64,9,0,0.333333,0.247738,2.839
2,question_conversational,float64,5,0,0.000000,0.889785,0.673
3,question_expect_short_answer,float64,5,0,1.000000,0.482974,1.958
4,question_fact_seeking,float64,5,0,1.000000,0.572463,1.647
5,question_has_commonly_accepted_answer,float64,5,0,1.000000,0.669683,1.524
6,question_interestingness_others,float64,9,0,0.666667,0.301201,2.462
7,question_interestingness_self,float64,9,0,0.333333,0.349400,2.559
8,question_multi_intent,float64,5,0,0.000000,0.594999,1.666
9,question_not_really_a_question,float64,5,0,0.000000,0.989143,0.100
